# Written map task: results

Juan Berrios | juanberrios@pitt.edu | Last updated: October 26, 2022

<b>Summary and overview of the data:</b>

- The purpose of this project is to do an analysis of the results of a map task being piloted for a study on code-switching. This pilot was conducted using only one language, either English or Spanish. In this first notebook I process the data, which is currently in a written chat log format.

**Contents:**

1. [Test run](#1.-Test-run): includes the necessary preparations, loading of files, and a test run of the relevant NLP operations on one sample script that is part of the full dataset.
2. [Processing](#2.-Processing): includes code for performing operations on the full set of transcript.
3. [Storing files](#3.-Storing-files): includes code for storing the results as `.pkl` and Excel spreadsheet files.

## 1. Test run

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import re, pickle

#Turning pretty print off:
%pprint

#Releasing all output:                                            
from IPython.core.interactiveshell import InteractiveShell #Prints all commands rather than the last one.
InteractiveShell.ast_node_interactivity = "all"

Pretty printing has been turned OFF


- We will now start with a test using only one of the `txt` files:

In [2]:
#Open file and extract lines as a list

fname = "./transcripts/1_06262021_A1B1.txt"   #Using transcript 1 as an example

with open(fname, encoding="utf-8-sig") as f:      #UTF-8 encoding included so special characters are kept.
    lines = f.read().splitlines()                 #Removes /n 
    lines = list(filter(None, lines))             #Filter out empty lines

In [3]:
lines[:10]  #sample

['9:48:23 pm - A1:', 'Start behind the surfer with the board overhead', 'Curve around the surfer’s feet', 'Then go towards the water at a slight angle, entering between the rock patches', '9:50:02 pm - B1:', 'Should I stop at the waters edge?', '9:50:09 pm - A1:', 'Nope', 'Slight curve to the blue water. At the level of the lady surfer’s head, make a straight line towards the pink dolphin', '9:51:14 pm - A1:']

In [4]:
#Create a data frame with the lines

df = pd.DataFrame(lines,columns=["turn"])

In [5]:
#sample (head and tail) and dimensions. 

df

,turn
0,9:48:23 pm - A1:
1,Start behind the surfer with the board overhead
2,Curve around the surfer’s feet
3,"Then go towards the water at a slight angle, e..."
4,9:50:02 pm - B1:
...,...
279,That’s the dewstination
280,Destination.
281,10:55:44 pm - B1:
282,Cool dewd


- An important first step in cleaning the data up is transposing the time/speaker rows that correspond to each turn. Once that is done then we can remove such rows. I'll  define some functions for this purpose, making use of regular expressions. Here's a [regex tester](https://regex101.com) with Python flavor, and [some documentation](https://www.guru99.com/python-regular-expressions-complete-tutorial.html) for the regex functions used.

In [6]:
#Defining functions

def get_time(turn):   
    """Takes a turn as input, returns time if the cell matches, returns 'prior' if it is a text cell"""
    match = re.match(r'\d+:\d+:\d+ (am|pm) - \D\d:', turn)
    if match:
        time = re.search(r'\d+:\d+:\d+ (am|pm)', match.group())
        return time.group()
    else:
        return 'prior'      

def get_participant(turn):   
    """Takes a turn as input, returns participant if the cell matches, returns 'prior' if it is a text cell"""
    match = re.match(r'\d+:\d+:\d+ (am|pm) - \D\d:', turn)
    if match:
        participant = re.search(r'\b\D\d\b', match.group())
        return participant.group()
    else:
        return 'prior'       

In [7]:
#Using functions on data frame to create new columns

df['time'] = df['turn'].map(get_time) 
df['participant'] = df['turn'].map(get_participant)

In [8]:
#Previewing results

df 

,turn,time,participant
0,9:48:23 pm - A1:,9:48:23 pm,A1
1,Start behind the surfer with the board overhead,prior,prior
2,Curve around the surfer’s feet,prior,prior
3,"Then go towards the water at a slight angle, e...",prior,prior
4,9:50:02 pm - B1:,9:50:02 pm,B1
...,...,...,...
279,That’s the dewstination,prior,prior
280,Destination.,prior,prior
281,10:55:44 pm - B1:,10:55:44 pm,B1
282,Cool dewd,prior,prior


In [9]:
#Adding columns containing other important information

df["language"] = "english" #Language of the sessions
df["session"] = "one"      #Number of the session
df["date"] = "06/26/2021"  #Date of the session

In [10]:
#Previewing results

df

,turn,time,participant,language,session,date
0,9:48:23 pm - A1:,9:48:23 pm,A1,english,one,06/26/2021
1,Start behind the surfer with the board overhead,prior,prior,english,one,06/26/2021
2,Curve around the surfer’s feet,prior,prior,english,one,06/26/2021
3,"Then go towards the water at a slight angle, e...",prior,prior,english,one,06/26/2021
4,9:50:02 pm - B1:,9:50:02 pm,B1,english,one,06/26/2021
...,...,...,...,...,...,...
279,That’s the dewstination,prior,prior,english,one,06/26/2021
280,Destination.,prior,prior,english,one,06/26/2021
281,10:55:44 pm - B1:,10:55:44 pm,B1,english,one,06/26/2021
282,Cool dewd,prior,prior,english,one,06/26/2021


In [11]:
#Deleting the sample data frame as it's no longer needed

del(df) 

## 2. Processing

In [12]:
#Master function to streamline processing

def transcript_process(fname, language, session, date):
    """Builds, cleans, and creates a data frame using the transcript file and
    keeping only the required rows. 'fname' corresponds to the file name of the session transcript,
    'language' corresponds to a choice between 'English and Spanish', 'session' corresponds to the number,
    and 'date' corresponds to the date the session was conducted. All the variables are strings"""
    with open(fname, encoding="utf-8-sig") as f:    
        lines = f.read().splitlines()              
        lines = list(filter(None, lines))
    df = pd.DataFrame(lines,columns=["turn"])
    df['time'] = df['turn'].map(get_time) 
    df['participant'] = df['turn'].map(get_participant) 
    df["language"] = language #Language of the sessions
    df["session"] = session      #Number of the session
    df["date"] = date  #Date of the sessions
    return df

- We will now use the function to process each of the transcripts:

In [13]:
#Going one by one here because I need to input different data for each. A dictionary could be used to streamline
#the process when working with a bigger dataset.

one = transcript_process("./transcripts/1_06262021_A1B1.txt", "English", "one", "07/06/2021")
two = transcript_process("./transcripts/2_06302021_C1D1.txt", "English", "two", "07/06/2021")
three = transcript_process("./transcripts/3_13072021_E1F1.txt", "English", "three", "07/13/2021")
four = transcript_process("./transcripts/4_07152021_G1H1.txt", "English", "four", "07/15/2021")
six = transcript_process("./transcripts/6_07222021_K1L1.txt", "English", "six", "07/22/2021")
seven = transcript_process("./transcripts/7_07232021_M1N1.txt", "Spanish", "seven", "07/23/2021")
eigth = transcript_process("./transcripts/8_07272021_O1P1.txt", "Spanish", "eight", "07/27/2021")
nine = transcript_process("./transcripts/9_07282021_Q1R1.txt", "Spanish", "nine", "07/28/2021")
ten = transcript_process("./transcripts/10_08042021_S1T1.txt", "English", "ten", "08/03/2021")
eleven = transcript_process("./transcripts/11_08192021_U1V1.txt", "English", "eleven", "08/19/2021")
twelve = transcript_process("./transcripts/12_09102021_W1X1.txt", "English", "twelve", "09/10/2021")
thirteen = transcript_process("./transcripts/13_05282022_Y1Z1.txt", "English", "thirteen", "09/10/2021")
fourteen = transcript_process("./transcripts/14_05282022_A2B2.txt", "English", "fourteen", "09/10/2021")
fifteen = transcript_process("./transcripts/15_06292022_C2D2.txt", "Spanish", "fifteen", "09/10/2021")
sixteen = transcript_process("./transcripts/16_07112022_E2F2.txt", "Spanish", "sixteen", "09/10/2021")
seventeen = transcript_process("./transcripts/17_09052022_G2H2.txt", "Spanish", "seventeen", "09/10/2021")
eighteen = transcript_process("./transcripts/18_07072022_I2J2.txt", "Spanish", "eighteen", "09/10/2021")
nineteen = transcript_process("./transcripts/19_07072022_K2L2.txt", "Spanish", "nineteen", "09/10/2021")
twenty = transcript_process("./transcripts/20_07072022_M2N2.txt", "Spanish", "twenty", "09/10/2021")

- Merging the processed data frames into a master data frame:

In [14]:
df = pd.concat([one, two, three, four, six, seven, eigth, nine, ten, eleven, twelve,thirteen,
               fourteen,fifteen,sixteen,eighteen,nineteen,twenty], sort=True)

In [15]:
#Previewing the columsn

df.keys() 

Index(['date', 'language', 'participant', 'session', 'time', 'turn'], dtype='object')

- The order of the columns was shuffled, to turn it back:

In [16]:
df = df[['turn', 'time', 'participant', 'language', 'session', 'date']]

In [17]:
#Previewing results

df 

,turn,time,participant,language,session,date
0,9:48:23 pm - A1:,9:48:23 pm,A1,English,one,07/06/2021
1,Start behind the surfer with the board overhead,prior,prior,English,one,07/06/2021
2,Curve around the surfer’s feet,prior,prior,English,one,07/06/2021
3,"Then go towards the water at a slight angle, e...",prior,prior,English,one,07/06/2021
4,9:50:02 pm - B1:,9:50:02 pm,B1,English,one,07/06/2021
...,...,...,...,...,...,...
195,ok listo,prior,prior,Spanish,twenty,09/10/2021
196,12:08:34 am - H2:,12:08:34 am,H2,Spanish,twenty,09/10/2021
197,ok,prior,prior,Spanish,twenty,09/10/2021
198,has terminado,prior,prior,Spanish,twenty,09/10/2021


- It looks good. However the index is wrong and also unnecessary

In [18]:
#Dropping index 

df.reset_index(drop=True, inplace=True)

In [19]:
df #Looks correct

,turn,time,participant,language,session,date
0,9:48:23 pm - A1:,9:48:23 pm,A1,English,one,07/06/2021
1,Start behind the surfer with the board overhead,prior,prior,English,one,07/06/2021
2,Curve around the surfer’s feet,prior,prior,English,one,07/06/2021
3,"Then go towards the water at a slight angle, e...",prior,prior,English,one,07/06/2021
4,9:50:02 pm - B1:,9:50:02 pm,B1,English,one,07/06/2021
...,...,...,...,...,...,...
3460,ok listo,prior,prior,Spanish,twenty,09/10/2021
3461,12:08:34 am - H2:,12:08:34 am,H2,Spanish,twenty,09/10/2021
3462,ok,prior,prior,Spanish,twenty,09/10/2021
3463,has terminado,prior,prior,Spanish,twenty,09/10/2021


- Let's try taking care of the "prior" values in order to get rid of the extraneous time/participant rows:

In [20]:
df = df.replace("prior", np.nan).ffill() #Replaces "prior" with Nan, and then Nan with the value just above it.
    
#More info on this solution: 
#https://stackoverflow.com/questions/47053712/conditional-change-of-a-pandas-row-with-the-previous-row-value

In [21]:
df #result

,turn,time,participant,language,session,date
0,9:48:23 pm - A1:,9:48:23 pm,A1,English,one,07/06/2021
1,Start behind the surfer with the board overhead,9:48:23 pm,A1,English,one,07/06/2021
2,Curve around the surfer’s feet,9:48:23 pm,A1,English,one,07/06/2021
3,"Then go towards the water at a slight angle, e...",9:48:23 pm,A1,English,one,07/06/2021
4,9:50:02 pm - B1:,9:50:02 pm,B1,English,one,07/06/2021
...,...,...,...,...,...,...
3460,ok listo,12:08:12 am,G2,Spanish,twenty,09/10/2021
3461,12:08:34 am - H2:,12:08:34 am,H2,Spanish,twenty,09/10/2021
3462,ok,12:08:34 am,H2,Spanish,twenty,09/10/2021
3463,has terminado,12:08:34 am,H2,Spanish,twenty,09/10/2021


- We can check that there are no 'prior' values left in the following way:

In [22]:
df.loc[df['time'] == 'prior']

,turn,time,participant,language,session,date


- As a last step, we will exclude rows beginning with a time format, which are no longer needed:

In [23]:
df = df[~df['turn'].str.contains(r'\d+:\d+:\d+', regex=True)] #removing rows
df.reset_index(drop=True, inplace=True) #resetting index

In [24]:
#Previewing results

df 

,turn,time,participant,language,session,date
0,Start behind the surfer with the board overhead,9:48:23 pm,A1,English,one,07/06/2021
1,Curve around the surfer’s feet,9:48:23 pm,A1,English,one,07/06/2021
2,"Then go towards the water at a slight angle, e...",9:48:23 pm,A1,English,one,07/06/2021
3,Should I stop at the waters edge?,9:50:02 pm,B1,English,one,07/06/2021
4,Nope,9:50:09 pm,A1,English,one,07/06/2021
...,...,...,...,...,...,...
2102,ahora haras una linea encima del tronco hasta ...,12:07:48 am,H2,Spanish,twenty,09/10/2021
2103,ok listo,12:08:12 am,G2,Spanish,twenty,09/10/2021
2104,ok,12:08:34 am,H2,Spanish,twenty,09/10/2021
2105,has terminado,12:08:34 am,H2,Spanish,twenty,09/10/2021


- Saving in different formats for later processing:

## 3. Storing files

In [25]:
df.to_excel("./spreadsheets/df.xlsx", index=False) #Saving as spreadsheet file
df.to_pickle('./pkl/df.pkl')                       #Pickling